## Import Libraries

In [1]:
%matplotlib inline
from os.path import expanduser, join
from time import time
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import r2_score

# from sklearn.externals import joblib
import joblib
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.linear_model import LinearRegression
import pandas as pd
from sklearn import ensemble

# from pyrallel.ensemble import EnsembleGrower
# from pyrallel.ensemble import sub_ensemble

# from ipyparallel import Client
# lb_view = Client().load_balanced_view()
# len(lb_view)

## Load and Read Data

In [2]:
df_train = pd.read_csv('data/Fold1/train.txt', sep=" ", header=None)
df_test = pd.read_csv('data/Fold1/test.txt', sep=" ", header=None)
df_valid = pd.read_csv('data/Fold1/vali.txt', sep=" ", header=None)

In [3]:
df_valid.head()

,0,1,2,3,4,5,6,7,8,9,...,129,130,131,132,133,134,135,136,137,138
0,0,qid:10,1:2,2:0,3:0,4:0,5:2,6:0.666667,7:0,8:0,...,128:1,129:0,130:117,131:55115,132:7,133:2,134:0,135:0,136:0,NaN
1,0,qid:10,1:1,2:0,3:1,4:3,5:3,6:0.333333,7:0,8:0.333333,...,128:0,129:0,130:153,131:3866,132:17,133:104,134:0,135:0,136:0,NaN
2,1,qid:10,1:3,2:0,3:3,4:0,5:3,6:1,7:0,8:1,...,128:0,129:9,130:266,131:56137,132:5,133:2,134:0,135:0,136:0,NaN
3,0,qid:10,1:3,2:0,3:2,4:0,5:3,6:1,7:0,8:0.666667,...,128:8,129:0,130:541,131:12621,132:11,133:11,134:0,135:0,136:0,NaN
4,1,qid:10,1:3,2:0,3:3,4:0,5:3,6:1,7:0,8:1,...,128:6,129:0,130:14687,131:40205,132:5,133:3,134:0,135:0,136:0,NaN


In [8]:
df_valid.iloc[0,1]

'qid:10'

In [5]:
def parse_mslr_dataset_line(line, map_fn_over_feature=None):
    """ 
    Compact Function to parse a single line from MSLR dataset txt file. 

    @args:
        line: str
        map_fn_over_features: fucntion to map over the extracted features

    @returns:
        Tuple[rel, qid, List[features]]
    """
    # Clean and split into array
    tokens = line.strip("\n").strip(" ").split(" ")

    # Lambda to parse out the val for qid
    extr_fn = lambda x: x.split(":")[-1]

    if map_fn_over_features is None:
        feat_fn = lambda x: str(x)
    else:
        feat_fn = map_fn_over_features
    # one-liner to extract and assign relevance, qid and features
    rel, qid, *features = \
    [int(extr_fn(c)) if idx < 2 else feat_fn(c) for idx, c in enumerate(tokens)]

    return rel, qid, features

import numpy as np
def clean(X):
    return [float(x.split(":")[-1]) if not isinstance(x, int) and not isinstance(x, float) else x for x in X]

In [9]:
df_train = df_train.apply(lambda x: clean(x))
df_train.drop([138],inplace=True,axis=1)

df_test = df_test.apply(lambda x: clean(x))
df_test.drop([138],inplace=True,axis=1)

df_valid = df_valid.apply(lambda x: clean(x))
df_valid.drop([138],inplace=True,axis=1)

In [11]:
df_valid.iloc[0,1]

10.0

In [ ]:
line = df_train.iloc[1,:].values

In [ ]:
# df_train[138].isnull().sum(axis = 0) - df_train.shape[0]

In [6]:
# Required Columns
required_columns = [0,15,20,25,30,35,40,45,75,80,85,90,95,105,110]
len(required_columns)

16

#### Total size in bytes, total number of search results and number of queries

In [ ]:
(df_train.values.nbytes + df_valid.values.nbytes + df_test.values.nbytes) / 1e6

In [ ]:
(df_train.shape[0] + df_valid.shape[0] + df_test.shape[0])

In [ ]:
len(np.unique(df_train.iloc[:,1])) + len(np.unique(df_test.iloc[:,1])) + len(np.unique(df_valid.iloc[:,1]))

### Concatenate the training and validation sets as a big development set.

In [ ]:
df = df_train.append(df_valid, ignore_index=True)
df.shape

In [ ]:
df.iloc[:,0].unique()

### Extract a subset of 500 queries to speed up the learning when prototyping

In [ ]:
def subsample(df, size, seed=None):
    rng = np.random.RandomState(seed)
    qid, X, y = df.iloc[:,1].values, df.drop([0,1],axis=1).values, df.iloc[:,0].values
    unique_qid = np.unique(qid)
    qid_mask = rng.permutation(len(unique_qid))[:size]
    subset_mask = np.in1d(qid, unique_qid[qid_mask])
    return X[subset_mask], y[subset_mask], qid[subset_mask]


X_train_small, y_train_small, qid_train_small = subsample(df, 500, seed=0)

Sanity check

In [ ]:
X_train_small.shape

In [ ]:
len(np.unique(qid_train_small))

In [ ]:
X_train_medium, y_train_medium, qid_train_medium = subsample(df, 1000, seed=0)

In [ ]:
def balance_irrelevant(X, y, qid, seed=None):
    """Subsample the zero-scored entries"""
    rng = np.random.RandomState(seed)
    unique_qid = np.unique(qid)
    final_mask = np.ones(shape=y.shape, dtype=bool)
    for this_qid in unique_qid:
        this_mask = qid == this_qid
        this_y = y[this_mask]
        relevant = this_y >= 2
        ratio = float(np.mean(relevant))
        if ratio > 0.5:
            # already balanced
            continue
            
        final_mask[this_mask] = np.logical_or(
            relevant, np.random.random(len(this_y)) > 0.7) 
    return X[final_mask], y[final_mask], qid[final_mask]

X_balanced_small, y_balanced_small, qid_balanced_small = balance_irrelevant(
    X_train_small, y_train_small, qid_train_small)

In [ ]:
print(len(y_train_small))
print(len(y_balanced_small))

### Quantifying ranking success with NDCG

In [ ]:
def dcg(relevances, rank=10):
    """Discounted cumulative gain at rank (DCG)"""
    relevances = np.asarray(relevances)[:rank]
    n_relevances = len(relevances)
    if n_relevances == 0:
        return 0.

    discounts = np.log2(np.arange(n_relevances) + 2)
    return np.sum(relevances / discounts)
 
 
def ndcg(relevances, rank=10):
    """Normalized discounted cumulative gain (NDGC)"""
    best_dcg = dcg(sorted(relevances, reverse=True), rank)
    if best_dcg == 0:
        return 0.

    return dcg(relevances, rank) / best_dcg

In [ ]:
ndcg([2, 4, 0, 1, 1, 0, 0], rank=5)

In [ ]:
ndcg([0, 0, 0, 1, 1, 2, 4], rank=5)

In [ ]:
ndcg([0, 0, 0, 1, 1, 2, 4], rank=3)

In [ ]:
ndcg([4, 2, 1, 1, 0, 0, 0], rank=5)

In [ ]:
def mean_ndcg(y_true, y_pred, query_ids, rank=10):
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    query_ids = np.asarray(query_ids)
    # assume query_ids are sorted
    ndcg_scores = []
    previous_qid = query_ids[0]
    previous_loc = 0
    for loc, qid in enumerate(query_ids):
        if previous_qid != qid:
            chunk = slice(previous_loc, loc)
            ranked_relevances = y_true[chunk][np.argsort(y_pred[chunk])[::-1]]
            ndcg_scores.append(ndcg(ranked_relevances, rank=rank))
            previous_loc = loc
        previous_qid = qid

    chunk = slice(previous_loc, loc + 1)
    ranked_relevances = y_true[chunk][np.argsort(y_pred[chunk])[::-1]]
    ndcg_scores.append(ndcg(ranked_relevances, rank=rank))
    return np.mean(ndcg_scores)


mean_ndcg([4, 3, 1, 4, 3], [4, 0, 1, 4, 2], [0, 0, 0, 2, 2], rank=10)

In [ ]:
def print_evaluation(model, X, y, qid):
    tic = time()
    y_predicted = model.predict(X)
    prediction_time = time() - tic
    print("Prediction time: {:.3f}s".format(prediction_time))
    print("NDCG@5 score: {:.3f}".format(
    mean_ndcg(y, y_predicted, qid, rank=5)))
    print("NDCG@10 score: {:.3f}".format(
    mean_ndcg(y, y_predicted, qid, rank=10)))
    print("NDCG score: {:.3f}".format(
    mean_ndcg(y, y_predicted, qid, rank=None)))
    print("R2 score: {:.3f}".format(r2_score(y, y_predicted)))

In [ ]:
def plot_ndcg_by_trees(model, X, y, qid, rank=10):
    max_n_trees = len(model.estimators_)
    scores = []
    
    if hasattr(model, 'staged_predict'):
        # stage-wise score computation for boosted ensembles
        n_trees = np.arange(max_n_trees) + 1
        for y_predicted in model.staged_predict(X):
            scores.append(mean_ndcg(y, y_predicted, qid, rank=10))
    else:
        # assume forest-type of tree ensemble: use a log scale to speedup
        # the computation
        # XXX: partial predictions could be reused
        n_trees = np.logspace(0, np.log10(max_n_trees), 10).astype(int)
        for j, n in enumerate(n_trees):
            y_predicted = ensemble(model, n).predict(X)
            scores.append(mean_ndcg(y, y_predicted, qid, rank=rank))
            
    plt.plot(n_trees, scores)
    plt.xlabel("Number of trees")
    plt.ylabel("Average NDCG@%d" % rank)
    _ = plt.title("Impact of the number of trees")

In [ ]:
%%time

from sklearn.ensemble import ExtraTreesRegressor

etr = ExtraTreesRegressor(n_estimators=200, min_samples_split=5, random_state=1, n_jobs=-1)
etr.fit(X_train_small, y_train_small)

In [ ]:
t=[2,3,4,5]
t[:None]

In [ ]:
X_test, y_test, qid_test = subsample(df_test, None, seed=0)
X_train, y_train, qid_train = subsample(df_train, None, seed=0)

In [ ]:
print_evaluation(etr, X_test, y_test, qid_test)

In [ ]:
print_evaluation(etr, X_train, y_train, qid_train)

In [ ]:
%%time

from sklearn.ensemble import RandomForestRegressor

rfr = RandomForestRegressor(n_estimators=200, min_samples_split=5, random_state=1, n_jobs=-1)
rfr.fit(X_balanced_small, y_balanced_small)

In [ ]:
print_evaluation(rfr, X_test, y_test, qid_test)

In [ ]:
%%time

from sklearn.ensemble import GradientBoostingRegressor

gbr = GradientBoostingRegressor(n_estimators=200, random_state=1, verbose=1)
gbr.fit(X_train_small, y_train_small)

In [ ]:
print_evaluation(gbr, X_test, y_test, qid_test)

### Comparing with a classification to NDCG ranking reduction models

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.base import RegressorMixin
from sklearn.preprocessing import StandardScaler
from sklearn.base import clone

In [ ]:
def proba_to_relevance(probas):
    """MCRank-like reduction of classification proba to DCG predictions"""
    rel = np.zeros(probas.shape[0], dtype=np.float32)
    for i in range(probas.shape[1]):
        rel += i * probas[:, i]
    return rel
        
        
class ClassificationRanker(RegressorMixin):
    
    def __init__(self, base_estimator=None):
        self.base_estimator = base_estimator
        
    def fit(self, X, y):
        self.estimator_ = clone(self.base_estimator)
        self.scaler_ = StandardScaler()
        X = self.scaler_.fit_transform(X)
        self.estimator_.fit(X, y)
        
    def predict(self, X):
        X_scaled = self.scaler_.transform(X)
        probas = self.estimator_.predict_proba(X_scaled)
        return proba_to_relevance(probas)

In [ ]:
%%time

from sklearn.ensemble import GradientBoostingClassifier

gbc = GradientBoostingClassifier(n_estimators=100, random_state=1)
gbr = ClassificationRanker(gbc)
gbr.fit(X_train_small, y_train_small)

In [ ]:
print_evaluation(gbr, X_test, y_test, qid_test)

In [ ]:
%%time

logr = ClassificationRanker(LogisticRegression(C=1000))
logr.fit(X_train_small, y_train_small)

In [ ]:
print_evaluation(logr, X_test, y_test, qid_test)

In [ ]:
%%time

from sklearn.ensemble import ExtraTreesClassifier

etc = ClassificationRanker(ExtraTreesClassifier(n_estimators=200, random_state=1, n_jobs=-1))
etc.fit(X_train_small, y_train_small)

In [ ]:
print_evaluation(etc, X_test, y_test, qid_test)